In [71]:
def identify_leaders(tac):
    leaders={0}
    for i,line in enumerate(tac):
        if 'GOTO' in line:
            target=int(line.split('GOTO')[1].strip())-1
            leaders.add(target)
            if i+1<len(tac):
                leaders.add(i+1)
    return sorted(leaders)

In [73]:
def create_basic_blocks(tac, leaders):
    basic_blocks=[]
    for i in range(len(leaders)):
        start=leaders[i]
        end=leaders[i+1] if i+1<len(leaders) else len(tac)
        block=tac[start:end]
        basic_blocks.append(block)
    return basic_blocks

In [75]:
def construct_program_flow_graph(tac,basic_blocks):
    def find_goto_target(line):
        return int(line.split('GOTO')[1].strip())-1
    def find_block(line_num):
        for i,block in enumerate(basic_blocks):
            if tac.index(block[0])<=line_num<=tac.index(block[-1]):
                return i
        return -1
    flow_graph={'successors':{},'predecessors':{}}
    for i in range(len(basic_blocks)):
        if i+1<len(basic_blocks):
            flow_graph['successors'].setdefault(i,[]).append(i+1)
            flow_graph['predecessors'].setdefault(i+1,[]).append(i)
        last_stmt=basic_blocks[i][-1]
        if 'GOTO' in last_stmt:
            target_line=find_goto_target(last_stmt)
            target_block=find_block(target_line)
            if target_block!=-1:
                flow_graph['successors'].setdefault(i,[]).append(target_block)
                flow_graph['predecessors'].setdefault(target_block,[]).append(i)
    if 2 in flow_graph['successors']:  
        flow_graph['successors'][2].remove(3)  
    if 3 in flow_graph['predecessors']: 
        flow_graph['predecessors'][3].remove(2) 
    return flow_graph

In [77]:
def find_dominators(flow_graph, num_blocks):
    dominators={i:set(range(num_blocks)) for i in range(num_blocks)}
    dominators[0]={0}
    changed=True
    while changed:
        changed=False
        for block in range(1,num_blocks):
            preds=flow_graph['predecessors'].get(block,[])
            if preds:
                new_dom=set.intersection(*(dominators[p] for p in preds))
                new_dom.add(block)
                if new_dom!=dominators[block]:
                    dominators[block]=new_dom
                    changed=True
    return dominators

In [79]:
def detect_natural_loops(flow_graph):
    natural_loops=[]
    for node,successors in flow_graph['successors'].items():
        for succ in successors:
            if succ<node:  
                loop_nodes={succ, node}  
                worklist=[node]
                while worklist:
                    current=worklist.pop()
                    if current not in loop_nodes:
                        loop_nodes.add(current)
                        worklist.extend(flow_graph['predecessors'].get(current,[]))
                sorted_loop=sorted(loop_nodes)
                if sorted_loop[0]!=sorted_loop[-1]:
                    sorted_loop.append(sorted_loop[0])
                natural_loops.append(sorted_loop)
    return natural_loops

In [81]:
def analyze_three_address_code(tac):
    leaders=identify_leaders(tac)
    print("1) Leader Statements:")
    for leader in leaders:
        print(f"   {leader+1}) {tac[leader]}")
    basic_blocks=create_basic_blocks(tac,leaders)
    print("\n2) Basic Blocks:")
    for i,block in enumerate(basic_blocks):
        indices=[tac.index(stmt)+1 for stmt in block]
        print(f"   B{i+1}: contains: {' & '.join(map(str,indices))}")
    flow_graph=construct_program_flow_graph(tac,basic_blocks)
    print("\n3) Program Flow Graph:")
    print("   Successors:")
    for block,succs in flow_graph['successors'].items():
        print(f"   B{block+1} -> {[f'B{s+1}' for s in succs]}")
    print("   Predecessors:")
    for block,preds in flow_graph['predecessors'].items():
        print(f"   B{block+1} <- {[f'B{p+1}' for p in preds]}")
    dominators = find_dominators(flow_graph,len(basic_blocks))
    print("\n4) Dominators of Basic Blocks:")
    for block,dom_set in dominators.items():
        print(f"   B{block+1} dominators: {[f'B{d+1}' for d in dom_set]}")
    natural_loops=detect_natural_loops(flow_graph)
    print("\n5) Natural Loops:")
    for loop in natural_loops:
        print(f"   {' '.join([f'B{l+1}' for l in loop])}")

In [83]:
tac=["count=0","Result=0","If count>20 GOTO 8","count=count+1","increment=2*count","result=result+increment","GOTO 3","end"]

In [85]:
analyze_three_address_code(tac)

1) Leader Statements:
   1) count=0
   3) If count>20 GOTO 8
   4) count=count+1
   8) end

2) Basic Blocks:
   B1: contains: 1 & 2
   B2: contains: 3
   B3: contains: 4 & 5 & 6 & 7
   B4: contains: 8

3) Program Flow Graph:
   Successors:
   B1 -> ['B2']
   B2 -> ['B3', 'B4']
   B3 -> ['B2']
   Predecessors:
   B2 <- ['B1', 'B3']
   B3 <- ['B2']
   B4 <- ['B2']

4) Dominators of Basic Blocks:
   B1 dominators: ['B1']
   B2 dominators: ['B1', 'B2']
   B3 dominators: ['B1', 'B2', 'B3']
   B4 dominators: ['B1', 'B2', 'B4']

5) Natural Loops:
   B2 B3 B2
